In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")


os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [3]:
## Loading the model and Embeddings

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings= GoogleGenerativeAIEmbeddings(model='models/embedding-001')

llm= ChatGoogleGenerativeAI(model='gemini-1.0-pro')

f:\Generative_AI\Projects\LangGraph-End_to_End\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
message=[{'role': 'system', 'content': 'you are helpful assistant'},
            {'role': 'user', 'content': 'hi how are you?'}]

result = llm.invoke(message)
print(result.content)

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 Developer instruction is not enabled for models/gemini-1.0-pro

In [5]:
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
        result = llm.invoke(self.message)
        return result.content

In [7]:
bot=Chatbot(system="you are helpful assistant")
bot("hi how are you?")

"I'm doing well, thank you!  As a large language model, I don't have feelings, but I'm ready to assist you. 😊  What can I help you with today? \n"

In [8]:
bot.message

[{'role': 'system', 'content': 'you are helpful assistant'},
 {'role': 'user', 'content': 'hi how are you?'},
 {'role': 'assistant',
  'content': "I'm doing well, thank you!  As a large language model, I don't have feelings, but I'm ready to assist you. 😊  What can I help you with today? \n"}]

In [10]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [11]:
prompt = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer related to fitness, health, or exercise.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 1800 * 0.2
Runs a calculation (e.g., calorie intake, BMI, workout progress) and returns the result.

fitness_wiki:
e.g. fitness_wiki: Benefits of Cardio
Returns a summary of fitness-related information from a fitness knowledge base (Wikipedia or similar).

exercise_plan:
e.g. exercise_plan: Beginner full-body workout
Returns a recommended workout plan based on the type of exercise or user level.

nutrition_info:
e.g. nutrition_info: Protein sources for vegetarians
Looks up nutritional information or suggestions for diets.

Example session:
Question: What are the benefits of cardio exercises?
Thought: I should look up information about the benefits of cardio exercises.
Action: fitness_wiki: Benefits of Cardio
PAUSE

You will be called again with this:

Observation: Cardio exercises improve cardiovascular health, increase lung capacity, and aid in weight loss.

You then output:

Answer: Cardio exercises improve cardiovascular health, increase lung capacity, and help with weight loss.
""".strip()

### Pattern Breakdown:

- ^: This matches the start of a string. It means the string must begin with what follows.
- Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
(\w+):
1. The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
2. \w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".
- :: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
(.*):
1. This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.
**What does it do?**
This regex is looking for a string that:

1. Starts with the text "Action:".
2. Has a word right after it, separated by a colon.
3. Then, after another colon, it captures everything that follow

In [12]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

In [13]:
# Testing the pattern
match1 = action_re.match(text1)
match1

<re.Match object; span=(0, 19), match='Action: Move: North'>

In [14]:
match1.group(1)

'Move'

In [15]:
match1.group(2)

'North'

In [16]:
match3 = action_re.match(text3)
match3

In [20]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [21]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [19]:
def calculate(number):
    return eval(number)

calculate("2+2")

4

In [22]:
calculate("20+2*5")

30

In [23]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [24]:
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [25]:
prompt

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop, you output an Answer related to fitness, health, or exercise.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 1800 * 0.2\nRuns a calculation (e.g., calorie intake, BMI, workout progress) and returns the result.\n\nfitness_wiki:\ne.g. fitness_wiki: Benefits of Cardio\nReturns a summary of fitness-related information from a fitness knowledge base (Wikipedia or similar).\n\nexercise_plan:\ne.g. exercise_plan: Beginner full-body workout\nReturns a recommended workout plan based on the type of exercise or user level.\n\nnutrition_info:\ne.g. nutrition_info: Protein sources for vegetarians\nLooks up nutritional information or suggestions for diets.\n\nExample session:\nQuestion: What ar

In [26]:
bot = Chatbot(prompt)

bot("hi how are you?")

'Thought: This is a greeting, not a question about fitness. I should respond appropriately. \nAction: None \nPAUSE \nObservation: N/A'

In [27]:
bot("what is your job?")

'Thought: This is a question about me, the AI. I should answer honestly.\nAction: None\nPAUSE\nObservation: N/A\n\nAnswer: I am an AI assistant, programmed to help with information and tasks. I can access and process information to answer your questions in a comprehensive and informative way, even if they are open ended, challenging, or strange. \n'

In [28]:

next_prompt="Tell me about quantum computing from Wikipedia."

In [29]:
next_prompt="Tell me about the current gdp of india from Wikipedia."

In [30]:
result=bot(next_prompt)
result

"Thought: I need to get information about India's GDP from Wikipedia.\nAction: None \nPAUSE\nObservation: N/A \n\nAnswer: I am sorry, I cannot access external websites or specific files online, including Wikipedia. My knowledge about India's current GDP is limited. To get the most up-to-date information, I recommend checking a reliable source like the World Bank or the International Monetary Fund websites. \n"

In [31]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
actions

[]

In [33]:
query("how are you?")

Thought: The user is asking about my well-being. I should respond politely, but I don't have feelings. 
Action: None 
PAUSE 



"Thought: The user is asking about my well-being. I should respond politely, but I don't have feelings. \nAction: None \nPAUSE \n"

In [34]:
query("Fifteen * twenty five")

Thought: I need to calculate the product of fifteen and twenty five.
Action: calculate: 15 * 25
PAUSE 

 -- running calculate 15 * 25
Observation: 375
Answer: Fifteen multiplied by twenty five is 375. 



'Answer: Fifteen multiplied by twenty five is 375. \n'

In [35]:
query("Has Simon written about AI?")

Thought: I need more information about who Simon is to determine if he has written about AI. 
Action: PAUSE 



'Thought: I need more information about who Simon is to determine if he has written about AI. \nAction: PAUSE \n'

In [36]:
print('Thought: Simon has written about AI and acknowledges the debate surrounding the term.\n\nAnswer: Yes, Simon has written about AI. \n')

Thought: Simon has written about AI and acknowledges the debate surrounding the term.

Answer: Yes, Simon has written about AI. 



In [37]:
query("can you tell me capital of usa?")

Thought: This question is not related to fitness, health, or exercise. However, I can still answer it. 
Action: None.
PAUSE 



'Thought: This question is not related to fitness, health, or exercise. However, I can still answer it. \nAction: None.\nPAUSE \n'